# idact - Prometheus sandbox

## Initial setup

Add `idact` to path:

In [1]:
import sys
import os
import bitmath
import getpass
import contextlib
import fabric
import logging
from pprint import pprint

def append_idact_path():
    idact_path = os.path.realpath(os.path.join(os.getcwd(), '../'))
    sys.path.append(idact_path)
append_idact_path()

from idact import *
from idact.detail.auth.set_password import set_password

os.environ['IDACT_KEY_LOCATION'] = os.path.join(os.getcwd(), '../.notebook-ssh')
try:
    os.mkdir(os.environ['IDACT_KEY_LOCATION'])
except OSError as e:
    print(e)

USER = 'plggarstka'

Hide debug information, setup context manager stack (for testing purposes)

## Add cluster (only first run)

In [2]:
cluster = add_cluster(name="pro",
                      user=USER,
                      host="pro.cyfronet.pl",
                      port=22,
                      auth=AuthMethod.PUBLIC_KEY,
                      key=KeyType.RSA,
                      install_key=True,
                      scratch="$SCRATCH")
save_environment('.idact-env')

2018-08-15 20:06:10 INFO: Generating public-private key pair.


## Load cluster (subsequent runs)

In [3]:
load_environment('.idact-env')
cluster = show_cluster("pro")
cluster

Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='E:\\shared\\uni\\eng-project\\notebooks\\../.notebook-ssh\\id_rsa_pl', install_key=True, disable_sshd=False)

In [4]:
set_log_level(logging.INFO)
#set_log_level(logging.DEBUG)
save_environment('.idact-env')

In [5]:
node = cluster.get_access_node()
node

Node(pro.cyfronet.pl:22, None)

In [6]:
node.run('whoami')

Password for plggarstka@pro.cyfronet.pl:22: 


'plggarstka'

In [7]:
node.run('hostname')

'login01.pro.cyfronet.pl'

## Allocate nodes

In [8]:
nodes = cluster.allocate_nodes(nodes=2,
                               cores=2,
                               memory_per_node=bitmath.GiB(10),
                               walltime=Walltime(minutes=20),
                               native_args={
                                   '--partition': 'plgrid-testing',
                                   '--account': 'intdata'
                               })

In [9]:
nodes

Nodes([Node(NotAllocated),Node(NotAllocated)])

In [10]:
nodes.wait()
nodes

Nodes([Node(p0638:50104, 2018-08-15 18:26:33.429904+00:00),Node(p0641:37379, 2018-08-15 18:26:33.429904+00:00)])

## Run commands

In [11]:
nodes[0].run('whoami')

'plggarstka'

In [12]:
nodes[0].run('hostname')

'p0638'

In [13]:
nodes[1].run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n          12592527 plgrid-te     wrap plggarst  R       0:06      2 p[0638,0641]'

In [14]:
nodes[1].run('hostname')

'p0641'

## Tunnel

In [15]:
tunnel = nodes[0].tunnel(here=9000, there=10000)

In [16]:
tunnel

MultiHopTunnel(9000:10000)

In [17]:
tunnel.close()

## Deploy notebook

One-time config step (cluster-specific):

In [18]:
cluster.config.setup_actions.jupyter = ['module load plgrid/tools/python-intel/3.6.2']
save_environment('.idact-env')

To run Jupyter Notebook on the cluster:

In [19]:
nb = nodes[0].deploy_notebook(local_port=8080)
nb

JupyterDeployment(8080 -> Node(p0638:50104, 2018-08-15 18:26:33.429904+00:00))

In [20]:
nb.local_port

8080

To open the deployed notebook server in a new tab:

In [21]:
nb.open_in_browser()

In [22]:
nb.cancel()

## Deploy Dask

One-time config step (cluster-specific):

In [23]:
cluster.config.setup_actions.dask = ['module load plgrid/tools/python-intel/3.6.2']
cluster.config.scratch = '$SCRATCH'
save_environment('.idact-env')

In [24]:
dd = deploy_dask(nodes)
dd

2018-08-15 20:07:16 INFO: Deploying scheduler...
2018-08-15 20:07:26 INFO: Deploying workers...
2018-08-15 20:07:26 INFO: Deploying worker 1/2...
2018-08-15 20:07:36 INFO: Deploying worker 2/2...


DaskDeployment(scheduler=tcp://localhost:64216/tcp://172.20.66.128:33683, workers=2)

Get Dask client:

In [25]:
client = dd.get_client()
client

Client Scheduler: tcp://localhost:64216 Dashboard: http://localhost:35373/status,Cluster Workers: 2 Cores: 4 Memory: 21.47 GB


Computation will work only if Python and library versions match:

In [26]:
#x = client.submit(lambda: value + 1, 10)
#x.result() == 11

Diagnostics servers are tunnelled:

In [27]:
dd.diagnostics.addresses

['http://localhost:35373', 'http://localhost:64227', 'http://localhost:64233']

To open diagnostics servers in new tabs:

In [28]:
dd.diagnostics.open_all()

In [29]:
dd.cancel()

## Close

In [30]:
nodes.running()

True

In [31]:
nodes.cancel()

In [32]:
nodes.running()

False

In [33]:
node.run('squeue')

'JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)'

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:64216 remote=tcp://localhost:64216>
